# ML match for LOFAR and the combined PanSTARRS WISE catalogue: Get all sources above the threshold

In this notebook the maximum likelihood cross-match between the LOFAR HETDEX catalogue and the combined PansSTARRS WISE catalogue is computed. However, we take all the sources that have likelihood ratios above the threshold found in the first run.

## Configuration

### Load libraries and setup

In [1]:
import numpy as np
from astropy.table import Table, join
from astropy import units as u
from astropy.coordinates import SkyCoord, search_around_sky
from IPython.display import clear_output
import pickle
import os

In [2]:
from mltier1 import (get_center, get_n_m, estimate_q_m, Field, SingleMLEstimator, MultiMLEstimator,
                     parallel_process, get_sigma, get_q_m)

In [3]:
%load_ext autoreload

In [4]:
%autoreload

In [5]:
from IPython.display import clear_output

In [6]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### General configuration

In [7]:
save_intermediate = True
plot_intermediate = True

In [8]:
idp = "idata/final"

In [9]:
if not os.path.isdir(idp):
    os.makedirs(idp)

### Area limits

In [10]:
# Busy week Edinburgh 2017
ra_down = 172.09
ra_up = 187.5833
dec_down = 46.106
dec_up = 56.1611

In [11]:
# Busy week Hatfield 2017
ra_down = 170.
ra_up = 190.
dec_down = 46.8
dec_up = 55.9

In [12]:
# Full field July 2017
ra_down = 160.
ra_up = 232.
dec_down = 42.
dec_up = 62.

In [13]:
field = Field(170.0, 190.0, 46.8, 55.9)

In [14]:
field_full = Field(160.0, 232.0, 42.0, 62.0)

## Load data

In [15]:
combined_all = Table.read("pw.fits")

In [16]:
lofar_all = Table.read("data/LOFAR_HBA_T1_DR1_catalog_v0.9.srl.fits")

In [17]:
np.array(combined_all.colnames)

array(['AllWISE', 'objID', 'ra', 'dec', 'raErr', 'decErr', 'W1mag',
       'W1magErr', 'i', 'iErr'],
      dtype='<U8')

In [18]:
np.array(lofar_all.colnames)

array(['Source_Name', 'RA', 'E_RA', 'E_RA_tot', 'DEC', 'E_DEC',
       'E_DEC_tot', 'Peak_flux', 'E_Peak_flux', 'E_Peak_flux_tot',
       'Total_flux', 'E_Total_flux', 'E_Total_flux_tot', 'Maj', 'E_Maj',
       'Min', 'E_Min', 'PA', 'E_PA', 'Isl_rms', 'S_Code', 'Mosaic_ID',
       'Isl_id'],
      dtype='<U16')

### Filter catalogues

In [19]:
lofar = field_full.filter_catalogue(lofar_all[lofar_all['Maj'] < 30.], 
                                         colnames=("RA", "DEC"))

In [20]:
combined = field_full.filter_catalogue(combined_all, 
                               colnames=("ra", "dec"))

### Additional data

In [21]:
combined["colour"] = combined["i"] - combined["W1mag"]

In [22]:
combined_aux_index = np.arange(len(combined))

### Sky coordinates

In [23]:
coords_combined = SkyCoord(combined['ra'], 
                           combined['dec'], 
                           unit=(u.deg, u.deg), 
                           frame='icrs')

In [24]:
coords_lofar = SkyCoord(lofar['RA'], 
                       lofar['DEC'], 
                       unit=(u.deg, u.deg), 
                       frame='icrs')

### Class of sources in the combined catalogue

The sources are grouped depending on the available photometric data.

In [25]:
combined_matched = (~np.isnan(combined["i"]) & ~np.isnan(combined["W1mag"])) # Matched i-W1 sources

In [26]:
combined_panstarrs = (~np.isnan(combined["i"]) & np.isnan(combined["W1mag"])) # Sources with only i-band

In [27]:
combined_wise =(np.isnan(combined["i"]) & ~np.isnan(combined["W1mag"])) # Sources with only W1-band

In [28]:
combined_i = combined_matched | combined_panstarrs
combined_w1 = combined_matched | combined_wise
#combined_only_i = combined_panstarrs & ~combined_matched
#combined_only_w1 = combined_wise & ~combined_matched

In [29]:
print("Total    - ", len(combined))
print("i and W1 - ", np.sum(combined_matched))
print("Only i   - ", np.sum(combined_panstarrs))
print("With i   - ", np.sum(combined_i))
print("Only W1  - ", np.sum(combined_wise))
print("With W1  - ", np.sum(combined_w1))

Total    -  31803022
i and W1 -  8196213
Only i   -  18583323
With i   -  26779536
Only W1  -  5023475
With W1  -  13219688


### Colour categories

The colour categories will be used after the first ML match

In [30]:
colour_limits = [-0.5, 0.0, 0.5, 1.0, 1.25, 1.5, 1.75, 2.0, 2.25, 2.5, 2.75, 3.0, 3.5, 4.0]

In [31]:
# Start with the W1-only, i-only and "less than lower colour" bins
colour_bin_def = [{"name":"only W1", "condition": combined_wise},
                  {"name":"only i", "condition": combined_panstarrs},
                  {"name":"-inf to {}".format(colour_limits[0]), 
                   "condition": (combined["colour"] < colour_limits[0])}]

# Get the colour bins
for i in range(len(colour_limits)-1):
    name = "{} to {}".format(colour_limits[i], colour_limits[i+1])
    condition = ((combined["colour"] >= colour_limits[i]) & 
                 (combined["colour"] < colour_limits[i+1]))
    colour_bin_def.append({"name":name, "condition":condition})

# Add the "more than higher colour" bin
colour_bin_def.append({"name":"{} to inf".format(colour_limits[-1]), 
                       "condition": (combined["colour"] >= colour_limits[-1])})

/disk2/jsm/prog/anaconda/envs/py36/lib/python3.6/site-packages/astropy/table/column.py:928: RuntimeWarning: invalid value encountered in less
  return getattr(self.data, oper)(other)
/disk2/jsm/prog/anaconda/envs/py36/lib/python3.6/site-packages/astropy/table/column.py:928: RuntimeWarning: invalid value encountered in greater_equal
  return getattr(self.data, oper)(other)


In [32]:
combined["category"] = np.nan
for i in range(len(colour_bin_def)):
    combined["category"][colour_bin_def[i]["condition"]] = i

In [33]:
np.sum(np.isnan(combined["category"]))

11

We get the number of sources of the combined catalogue in each colour category. It will be used at a later stage to compute the $Q_0$ values

In [34]:
numbers_combined_bins = np.array([np.sum(a["condition"]) for a in colour_bin_def])

In [35]:
numbers_combined_bins

array([ 5023475, 18583323,   617057,   435585,   680625,   914717,
         656883,   775440,   830399,   803840,   712001,   578606,
         437076,   307929,   317143,    94996,    33916])

## Maximum Likelihood

In [36]:
bin_list, centers, Q_0_colour, n_m, q_m = pickle.load(open("lofar_params.pckl", "rb"))

In [37]:
likelihood_ratio_function = MultiMLEstimator(Q_0_colour, n_m, q_m, centers)

### ML match

In [38]:
radius = 15

In [39]:
lr_threshold = 0.36

In [40]:
selection = ~np.isnan(combined["category"]) # Avoid the dreaded sources with no actual data
catalogue = combined[selection]

In [41]:
def apply_ml(i, likelihood_ratio_function):
    idx_0 = idx_i[idx_lofar == i]
    d2d_0 = d2d[idx_lofar == i]
    
    category = catalogue["category"][idx_0].astype(int)
    mag = catalogue["i"][idx_0]
    mag[category == 0] = catalogue["W1mag"][idx_0][category == 0]
    
    lofar_ra = lofar[i]["RA"]
    lofar_dec = lofar[i]["DEC"]
    lofar_pa = lofar[i]["PA"]
    lofar_maj_err = lofar[i]["E_Maj"]
    lofar_min_err = lofar[i]["E_Min"]
    c_ra = catalogue["ra"][idx_0]
    c_dec = catalogue["dec"][idx_0]
    c_ra_err = catalogue["raErr"][idx_0]
    c_dec_err = catalogue["decErr"][idx_0]
    
    sigma = get_sigma(lofar_maj_err, lofar_min_err, lofar_pa, 
                      lofar_ra, lofar_dec, 
                      c_ra, c_dec, c_ra_err, c_dec_err)

    lr_0 = likelihood_ratio_function(mag, d2d_0.arcsec, sigma, category)
    
    chosen_index = (lr_0 >= lr_threshold)
    result = [combined_aux_index[selection][idx_0[chosen_index]], # Index
              (d2d_0.arcsec)[chosen_index],                        # distance
              lr_0[chosen_index]]                                  # LR
    return result

### Run the cross-match

In [42]:
idx_lofar, idx_i, d2d, d3d = search_around_sky(
    coords_lofar, coords_combined[selection], radius*u.arcsec)

In [43]:
idx_lofar_unique = np.unique(idx_lofar)

In [48]:
total_sources = len(idx_lofar_unique)

### Run the ML matching

In [ ]:
def ml(i):
    return apply_ml(i, likelihood_ratio_function)

In [ ]:
res = parallel_process(idx_lofar_unique, ml, n_jobs=8)

 34%|███▍      | 105K/308K [56:59<1:52:57, 30.0it/s]  

### Selection and match

In [63]:
lofar_aux_index = np.arange(len(lofar))

In [73]:
lr = []
lr_dist = []
lr_index = []
lr_order = []
lr_lofar_index = []

for i, idx in enumerate(idx_lofar_unique):
    result = res[i]
    n = len(result[0])
    lofar_index = lofar_aux_index[idx]
    if n > 0:
        order = np.argsort(result[2])[::-1]
        lr.extend(result[2][order])
        lr_dist.extend(result[1][order])
        lr_index.extend(result[0][order])
        lr_order.extend(np.arange(n, dtype=int) + 1)
        lr_lofar_index.extend(np.ones(n, dtype=int)*lofar_index)
    else:
        lr.append(np.nan)
        lr_dist.append(np.nan)
        lr_index.append(np.nan)
        lr_order.append(np.nan)
        lr_lofar_index.append(lofar_index)

In [74]:
aux_table = Table()
aux_table['aux_index'] = lr_lofar_index
aux_table['lr'] = lr
aux_table['lr_dist'] = lr_dist
aux_table['lr_index'] = lr_index
aux_table['lr_order'] = lr_order

In [75]:
aux_table

aux_index,lr,lr_dist,lr_index,lr_order
int64,float64,float64,float64,float64
0,1245.71683876,0.256393266029,10944453.0,1.0
1,364.107736672,0.55383604885,10862628.0,1.0
2,nan,nan,nan,nan
3,1540.26629749,0.212102609363,10844459.0,1.0
4,532.351012101,0.300592058961,10817466.0,1.0
5,nan,nan,nan,nan
6,616.03186109,0.181201605232,10989213.0,1.0
7,113.330702705,1.2520854396,10850543.0,1.0
8,271.457376431,1.20606318915,11011954.0,1.0


In [76]:
lofar["aux_index"] = lofar_aux_index

In [77]:
lofar_lr = join(lofar, aux_table, join_type='outer', keys='aux_index')

In [79]:
lofar_lr

Source_Name,RA,E_RA,E_RA_tot,DEC,E_DEC,E_DEC_tot,Peak_flux,E_Peak_flux,E_Peak_flux_tot,Total_flux,E_Total_flux,E_Total_flux_tot,Maj,E_Maj,Min,E_Min,PA,E_PA,Isl_rms,S_Code,Mosaic_ID,Isl_id,aux_index,lr,lr_dist,lr_index,lr_order
,deg,arcsec,arcsec,deg,arcsec,arcsec,mJy / beam,mJy / beam,mJy / beam,mJy,mJy,mJy,arcsec,arcsec,arcsec,arcsec,deg,deg,mJy / beam,,,,,,,,
str24,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str1,str8,int32,int64,float64,float64,float64,float64
ILTJ122108.441+491340.94,185.285168771,0.0850700841174,0.150016049008,49.2280397678,0.094025008434,0.155269436656,1.48718776646,0.0525447165803,0.30204311836,1.45167723509,0.0918727162887,0.304524658761,6.19600662707,0.228961053509,5.67333350682,0.191652766315,153.935376162,18.4908886458,0.0529464996362,S,P22Hetde,188,0,1245.71683876,0.256393266029,10944453.0,1.0
ILTJ122107.851+491951.84,185.282710884,0.473938135007,0.489780819921,49.3310690203,0.474978105451,0.490787221111,0.494206901892,0.0658548772285,0.118770717476,0.847902990364,0.0975490731081,0.195635888597,9.12706565844,1.35780957336,6.76932220205,0.807971003466,46.4592997429,23.1085152836,0.0616828474449,S,P22Hetde,196,1,364.107736672,0.55383604885,10862628.0,1.0
ILTJ122108.572+493121.89,185.285716341,0.533848043611,0.547961339344,49.52274915,0.367264254936,0.387493133749,0.357484003783,0.0532574812958,0.0891524078889,0.438294187244,0.087484556291,0.123845142708,7.79035197589,1.33867766644,5.66738515187,0.732001245106,67.3769455634,23.8529803004,0.0519686545886,S,P22Hetde,202,2,nan,nan,nan,nan
ILTJ122104.027+490823.80,185.266778578,0.0680609727689,0.141068039479,49.1399451347,0.0689599603528,0.141503964185,2.18501193876,0.0561302300848,0.440592430292,2.39268273523,0.0943671437026,0.487752380486,6.30982940769,0.162893477077,6.24977288427,0.159758583039,158.418653032,107.214019258,0.0549563665118,S,P22Hetde,204,3,1540.26629749,0.212102609363,10844459.0,1.0
ILTJ122105.030+491440.01,185.270960099,0.585020092161,0.597926754695,49.2444492774,0.508482783423,0.523280648205,0.291824013394,0.0552949639572,0.0803989006813,0.350985385828,0.0906813300159,0.114676646515,7.2605736876,1.50440230517,5.96540847045,1.03365648739,125.162791534,44.7934049203,0.0535216240678,S,P22Hetde,205,4,532.351012101,0.300592058961,10817466.0,1.0
ILTJ122106.722+492420.20,185.278009798,0.476205256172,0.491974940169,49.4056116521,0.363568823749,0.383992428767,0.308482936742,0.0504988664517,0.0797283161953,0.306304270094,0.0879554002244,0.107186961258,6.40838974909,1.12431943609,5.57965677846,0.852251276603,85.3961636616,52.8570913248,0.0507702061441,S,P22Hetde,206,5,nan,nan,nan,nan
ILTJ122106.172+492908.04,185.275717593,0.183850706166,0.221515186626,49.4855672648,0.257120714935,0.285269973528,0.641852509513,0.0532738711074,0.138985938503,0.671735160982,0.091377926785,0.162477846378,6.73049491464,0.611268985317,5.59946675829,0.424736261495,12.4682229602,21.5508992193,0.0530273173354,S,P22Hetde,210,6,616.03186109,0.181201605232,10989213.0,1.0
ILTJ122104.649+492207.60,185.269372038,0.515425109037,0.530029186719,49.3687791357,0.37925775751,0.398878856772,0.349770333809,0.0544436946502,0.088643597308,0.358176497234,0.0955404857905,0.119413569379,7.27808684182,1.354813951,5.06678218192,0.659476070658,122.170405666,22.0608157459,0.0554672114959,S,P22Hetde,211,7,113.330702705,1.2520854396,10850543.0,1.0


## Save combined catalogue

In [80]:
combined["lr_index"] = combined_aux_index.astype(float)

In [82]:
for col in ['lr', 'lr_dist', 'lr_index', 'lr_order']:
    lofar_lr[col].fill_value = np.nan

In [85]:
pwl = join(lofar_lr.filled(), combined, join_type='left', keys='lr_index')

In [88]:
len(pwl)

369243

In [86]:
pwl_columns = pwl.colnames

In [89]:
for col in pwl_columns:
    fv = pwl[col].fill_value
    #print(col, fv)
    if (isinstance(fv, np.float64) and (fv != 1e+20)):
        print(col, fv)
        pwl[col].fill_value = 1e+20

dec 1.0
W1mag 1.0
i 1.0
colour 1.0
category 1.0


In [91]:
columns_save = ['Source_Name', 'RA', 'E_RA', 'E_RA_tot', 'DEC', 'E_DEC', 'E_DEC_tot',
 'Peak_flux', 'E_Peak_flux', 'E_Peak_flux_tot', 'Total_flux', 'E_Total_flux', 'E_Total_flux_tot',
 'Maj', 'E_Maj', 'Min', 'E_Min', 'PA', 'E_PA', 'Isl_rms', 'S_Code', 'Mosaic_ID', 'Isl_id',
 'AllWISE', 'objID', 'ra', 'dec', 'raErr', 'decErr',
 'W1mag', 'W1magErr', 'i', 'iErr', 'colour', 'category',
 'lr', 'lr_dist', 'lr_order']

In [92]:
pwl[columns_save].write('lofar_multi_lr_pw.fits', format="fits")